In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [2]:
class InceptionA(nn.Module):

    def __init__(self, in_channels):
        super(InceptionA, self).__init__()
        self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

        self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
        self.branch3x3dbl_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)

        self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)

        outputs = [branch1x1, branch5x5, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(88, 20, kernel_size=5)

        self.incept1 = InceptionA(in_channels=10)
        self.incept2 = InceptionA(in_channels=20)

        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(1408, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = self.incept1(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = self.incept2(x)
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [17]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [18]:
for epoch in range(1, 10):
    train(epoch)
    test()

<ipython-input-3-497de6128dd0>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.303285
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.319355
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.285828
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.303838
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.286091
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.277684
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.284183
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.273488
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.287369
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.247822
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.250272
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.216012
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.180475
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.114498
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.076052
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.887848
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.740558
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.289450
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.250345
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.788795
Train Epoch: 1 [12800/60000 (

<ipython-input-17-8c4a7a4049bc>:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Test set: Average loss: 0.1990, Accuracy: 9391/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.247237
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.181091
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.188978
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.275983
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.251055
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.205897
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.144759
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.262729
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.179781
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.126404
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.104850
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.459838
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.179840
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.064107
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.256546
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.201195
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.254656
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.167334
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.133541
Train Epoch: 2 [12

Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.101191
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.091580
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.093065
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.074743
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.080265
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.089433
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.113443
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.013428
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.086899
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.084067
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.034209
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.034209
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.143794
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.049640
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.059785
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.067786
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.080231
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.107767
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.063889
Train Epoch: 3 [56320/60000 (94%)]	Loss: 0.121821


Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.149361
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.045917
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.088885
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.036152
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.102900
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.010354
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.095802
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.090711
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.094083
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.188331
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.016817
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.272082
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.043221
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.097102
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.114022
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.248810
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.042055
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.059684
Train Epoch: 5 [39680/60000 (66%)]	Loss: 0.033464
Train Epoch: 5 [40320/60000 (67%)]	Loss: 0.049308


Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.033011
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.017610
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.011902
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.028935
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.014650
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.065207
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.053181
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.112738
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.121465
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.016990
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.023640
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.049014
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.152948
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.008192
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.033065
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.007795
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.081210
Train Epoch: 7 [23040/60000 (38%)]	Loss: 0.022604
Train Epoch: 7 [23680/60000 (39%)]	Loss: 0.100096
Train Epoch: 7 [24320/60000 (41%)]	Loss: 0.078694


Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.018995
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.066719
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.105990
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.005882
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.327029

Test set: Average loss: 0.0650, Accuracy: 9781/10000 (98%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.007606
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.132898
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.021799
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.034203
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.034610
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.038565
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.078967
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.024526
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.010417
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.039346
Train Epoch: 9 [6400/60000 (11%)]	Loss: 0.041507
Train Epoch: 9 [7040/60000 (12%)]	Loss: 0.174870
Train Epoch: 9 [7680/60000 (13%)]	Loss: 0.012355
Train Epoch: 9 [8320/60000 (14%)]	Loss: 0.045106
Train Epoch: 9 [